# Data processing

# Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from constants import RANDOM_FOREST_PARAMS, NUM_FEATURES, CAT_FEATURES

from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

from dask import delayed, compute
from dask.diagnostics import ProgressBar

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

from category_encoders import BinaryEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

import optuna
from optuna.samplers import TPESampler
from sklearn.svm import SVC

# Data Loading

In [2]:
X_list = []
y_list = []

for i in range(10):
    X = pd.read_parquet(f"../../../../data/interim/copper/X_train_bal/X_{i}.parquet")
    y = pd.read_parquet(f"../../../../data/interim/copper/y_train_bal/y_{i}.parquet")

    X_list.append(X)
    y_list.append(y)

X_train = pd.read_parquet("../../../../data/interim/copper/X_train.parquet")
y_train = pd.read_parquet("../../../../data/interim/copper/y_train_cat.parquet")

# Tunnig Random Forest


## Setting Up Pipeline Steps

In [3]:
over = SMOTE(sampling_strategy="auto")
under = RandomUnderSampler(sampling_strategy="auto")

numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('imputer', KNNImputer(n_neighbors=5)),
    ])

categorical_transformer = Pipeline(steps=[
    ('binary_encoder', BinaryEncoder())
])

preprocessor = ColumnTransformer(
        transformers=[
            ('num_pca', numeric_transformer, NUM_FEATURES),
            ('cat', categorical_transformer, CAT_FEATURES)
        ]
    )

rkf= RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=42)

## Optimization

In [11]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

def objective(trial):
    params = dict()
    params['n_estimators'] = trial.suggest_int('n_estimators', 50, 500)
    params['learning_rate'] = trial.suggest_float('learning_rate', 0.01, 0.5)
    params['algorithm'] = trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R'])
    
    base_estimator = DecisionTreeClassifier(max_depth=trial.suggest_int('max_depth', 1, 10))
    
    steps_list = [('preprocessor', preprocessor),
                  ('over', over),
                  ('under', under),
                  ('classifier', AdaBoostClassifier(estimator=base_estimator, **params))]
    
    pipe = ImbPipeline(steps_list)
    
    scores = cross_val_score(pipe, X_train, y_train, cv=rkf, scoring="roc_auc")
    
    return np.mean(scores), np.sqrt(np.std(scores))

study = optuna.create_study(directions=['maximize', 'minimize'], sampler=TPESampler())
study.optimize(objective, n_trials=1500)

[I 2024-10-28 03:12:21,862] A new study created in memory with name: no-name-831067e6-533f-41c8-a49a-e94ef307cfe9
[I 2024-10-28 03:12:22,408] Trial 0 finished with values: [0.5239756367663344, 0.29843822812987764] and parameters: {'n_estimators': 378, 'learning_rate': 0.2652054035211346, 'algorithm': 'SAMME.R', 'max_depth': 10}.
[I 2024-10-28 03:12:22,974] Trial 1 finished with values: [0.5158568660022149, 0.29741846049973386] and parameters: {'n_estimators': 106, 'learning_rate': 0.24816146557373597, 'algorithm': 'SAMME', 'max_depth': 10}.
[I 2024-10-28 03:12:23,788] Trial 2 finished with values: [0.5010243632336655, 0.3452318537150911] and parameters: {'n_estimators': 118, 'learning_rate': 0.04800768656247927, 'algorithm': 'SAMME', 'max_depth': 5}.
[I 2024-10-28 03:12:24,410] Trial 3 finished with values: [0.4971345514950166, 0.2622023017335727] and parameters: {'n_estimators': 73, 'learning_rate': 0.1844880085661881, 'algorithm': 'SAMME', 'max_depth': 10}.
[I 2024-10-28 03:12:24,954

In [9]:
optuna.visualization.plot_pareto_front(study, target_names=["mean ROC-AUC", "std ROC-AUC"])

In [31]:
study_df = study.trials_dataframe()
study_df[study_df['number'] == 345]

number  values_0  values_1             datetime_start  \
345     345  0.716234  0.048805 2024-10-23 21:31:34.781092   

             datetime_complete               duration  params_max_depth  \
345 2024-10-23 21:31:35.513954 0 days 00:00:00.732862                 5   

     params_max_features  params_min_samples_leaf  params_min_samples_split  \
345                    1                        4                        50   

     params_n_estimators  params_random_state     state  
345                    6                   17  COMPLETE

In [35]:
#study_df.to_parquet("../../../../data/processed/copper/optuna_results_random_forest_raw_train.parquet")  # Save the results to a parquet file

# Seleção de features com Boruta

In [5]:
def importance_extimation(X, y, i):
    model = RandomForestClassifier(RANDOM_FOREST_PARAMS)

    feat_selector = BorutaPy(model, n_estimators="auto", random_state=i)

    feat_selector.fit(X, y)

    return feat_selector.ranking_

@delayed
def importance_experiment(X_list, y_list, i):
    
    rankings_list = [importance_extimation(X.values, y.values, i) for X, y in zip(X_list, y_list)]

    return rankings_list

In [6]:
# Create a list of delayed tasks
tasks = [importance_experiment(X_list, y_list, i) for i in range(5)]

# Use Dask to compute the results in parallel
with ProgressBar():
    experiments = compute(*tasks)

[                                        ] | 0% Completed | 349.13 us

[########################################] | 100% Completed | 348.79 s


In [7]:
exp_dfs = []

for exp in experiments:
    exp_dfs.append(pd.DataFrame(exp, columns=X.columns).reset_index())

exp_df = pd.concat(exp_dfs)
exp_df.reset_index(drop=True, inplace=True)

exp_median_df = exp_df.groupby("index").median()

plot_df = (
    exp_median_df.T
    .reset_index()
    .rename(columns={"index": "feature"})
    .melt(id_vars="feature", var_name="experiment", value_name="rank")
)

order = plot_df.groupby("feature")["rank"].mean().sort_values().index

In [8]:
plt.figure(figsize=(3, 7))
sns.barplot(plot_df, y = "feature", x = "rank", orient="h", order=order)

<Axes: xlabel='rank', ylabel='feature'>